In [43]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import json
import time
import gmaps

# Import API key
from api_keys import g_key

In [44]:
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum Temperature,Wind Speed
0,Jamestown,0,US,1646454527,56,42.0970,-79.2353,27.34,3.44
1,Bluff,100,NZ,1646454527,70,-46.6000,168.3333,71.80,8.59
2,Bredasdorp,12,ZA,1646454528,94,-34.5322,20.0403,61.29,1.14
3,Chokurdakh,38,RU,1646454529,99,70.6333,147.9167,-19.28,1.21
4,Sokolo,91,ML,1646454529,13,14.7328,-6.1219,72.88,9.17
...,...,...,...,...,...,...,...,...,...
572,Yashkino,100,RU,1646454737,91,55.8736,85.4265,18.55,8.52
573,Nyurba,100,RU,1646454828,86,63.2842,118.3319,7.38,5.06
574,Sola,31,VU,1646454829,71,-13.8833,167.5500,82.67,4.03
575,Yeppoon,43,AU,1646454829,72,-23.1333,150.7333,92.98,10.09


In [45]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Latitude", "Longitude"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [47]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [48]:
max_intensity

100

In [49]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Maximum Temperature"] >= 70) & (weather_data["Maximum Temperature"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum Temperature,Wind Speed
88,Bagé,0,BR,1646454572,88,-31.3314,-54.1069,71.38,5.75
176,Jupiter,0,US,1646454624,73,26.9342,-80.0942,75.00,8.99
215,Ixtapa,0,MX,1646454643,82,20.7000,-105.2000,70.23,1.01
232,Pisco,0,PE,1646454651,88,-13.7000,-76.2167,70.02,8.05
289,Bārmer,0,IN,1646454683,23,25.7500,71.3833,78.13,6.49
390,Kasane,0,BW,1646454735,100,-17.8167,25.1500,71.69,3.44
484,Salalah,0,OM,1646454783,37,17.0151,54.0924,71.69,2.30
559,Brownsville,0,US,1646454732,87,25.9018,-97.4975,70.92,9.22


In [50]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Latitude", "Longitude"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
88,Bagé,BR,-31.3314,-54.1069,
176,Jupiter,US,26.9342,-80.0942,
215,Ixtapa,MX,20.7000,-105.2000,
232,Pisco,PE,-13.7000,-76.2167,
289,Bārmer,IN,25.7500,71.3833,
390,Kasane,BW,-17.8167,25.1500,
484,Salalah,OM,17.0151,54.0924,
559,Brownsville,US,25.9018,-97.4975,


In [51]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [52]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Record does not exist; searching for next record...")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 88: Bagé.
Closest hotel in Bagé is Hotel Villa de Carli.
------------
Retrieving Results for Index 176: Jupiter.
Closest hotel in Jupiter is Wyndham Grand Jupiter At Harbourside Place.
------------
Retrieving Results for Index 215: Ixtapa.
Closest hotel in Ixtapa is Flamingo Vallarta Hotel & Marina.
------------
Retrieving Results for Index 232: Pisco.
Closest hotel in Pisco is Hostal El Candelabro.
------------
Retrieving Results for Index 289: Bārmer.
Closest hotel in Bārmer is Sanchal Fort.
------------
Retrieving Results for Index 390: Kasane.
Closest hotel in Kasane is Chobe Safari Lodge.
------------
Retrieving Results for Index 484: Salalah.
Closest hotel in Salalah is Salalah Gardens Hotel.
------------
Retrieving Results for Index 559: Brownsville.
Closest hotel in Brownsville is Hotel Ritz.
------------
-------End of Search-------


In [53]:
# Display the hotel dataframe
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
88,Bagé,BR,-31.3314,-54.1069,Hotel Villa de Carli
176,Jupiter,US,26.9342,-80.0942,Wyndham Grand Jupiter At Harbourside Place
215,Ixtapa,MX,20.7000,-105.2000,Flamingo Vallarta Hotel & Marina
232,Pisco,PE,-13.7000,-76.2167,Hostal El Candelabro
289,Bārmer,IN,25.7500,71.3833,Sanchal Fort
390,Kasane,BW,-17.8167,25.1500,Chobe Safari Lodge
484,Salalah,OM,17.0151,54.0924,Salalah Gardens Hotel
559,Brownsville,US,25.9018,-97.4975,Hotel Ritz


In [54]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [55]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))